In [40]:
import pandas as pd
import random
import os
import numpy as np
from math import *

## 결측 위한 library - MICE
from fancyimpute import IterativeImputer

# scalar
from sklearn.preprocessing import LabelEncoder

## Regression / Classifier 모델
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor

In [41]:
class cfg:
    seed=37
def seed_(seed):
    random.seed(seed)
    os.environ['PYTONHASHSEED'] = str(seed)
    np.random.seed(seed=seed)
seed_(cfg.seed)

In [42]:
# 데이터 불러오기
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

In [43]:
# TIMESTAMP DROP
train = train.drop(['TIMESTAMP'], axis=1)
test = test.drop(['TIMESTAMP'], axis=1)

In [44]:
# LINE별로 분리 해주세용
train_T050304 = train[train['LINE']=='T050304'] # 78
train_T050307 = train[train['LINE']=='T050307'] # 42
train_T100304 = train[train['LINE']=='T100304'] # 175
train_T100306 = train[train['LINE']=='T100306'] # 174
train_T010306 = train[train['LINE']=='T010306'] # 70
train_T010305 = train[train['LINE']=='T010305'] # 59

test_T050304 = test[test['LINE']=='T050304']
test_T050307 = test[test['LINE']=='T050307']
test_T100304 = test[test['LINE']=='T100304']
test_T100306 = test[test['LINE']=='T100306']
test_T010306 = test[test['LINE']=='T010306']
test_T010305 = test[test['LINE']=='T010305']

print('LINE Split Done.')

LINE Split Done.


In [45]:
notwith1 = []
notwith2 = []
notwith3 = []
notwith4 = []
notwith5 = []
notwith6 = []

# product별 사용하지 않는 변수 확인
for col in train_T050304.columns:
    if pd.notnull(train_T050304[col]).sum()==0:
        notwith1.append(col)

for col in train_T050307.columns:
    if pd.notnull(train_T050307[col]).sum()==0:
        notwith2.append(col)

for col in train_T100304.columns:
    if pd.notnull(train_T100304[col]).sum()==0:
        notwith3.append(col)
        
for col in train_T100306.columns:
    if pd.notnull(train_T100306[col]).sum()==0:
        notwith4.append(col)

for col in train_T010306.columns:
    if pd.notnull(train_T010306[col]).sum()==0:
        notwith5.append(col)

for col in train_T010305.columns:
    if pd.notnull(train_T010305[col]).sum()==0:
        notwith6.append(col)

print('All NaN Check.')

# 확인하고 제거
for col in notwith1:
    train_T050304 = train_T050304.drop([col], axis=1) # A_31 + T050304 , 1973 - 아래와는 1961 겹침 , 같은 A라인들과 765겹침
    test_T050304 = test_T050304.drop([col], axis=1)

for col in notwith2:
    train_T050307 = train_T050307.drop([col], axis=1) # A_31 + T050307 , 1980
    test_T050307 = test_T050307.drop([col], axis=1)

for col in notwith3:
    train_T100304 = train_T100304.drop([col], axis=1) # T(172) + O(3) + T100304 , 676 - 아래와는 670 겹침
    test_T100304 = test_T100304.drop([col], axis=1)

for col in notwith4:
    train_T100306 = train_T100306.drop([col], axis=1) # T(171) + O(3) + T100306 , 676
    test_T100306 = test_T100306.drop([col], axis=1)

for col in notwith5:
    train_T010306 = train_T010306.drop([col], axis=1) # A_31 + T010306 , 891 - 아래와는 883 겹침
    test_T010306 = test_T010306.drop([col], axis=1)

for col in notwith6:
    train_T010305 = train_T010305.drop([col], axis=1) # A_31 + T010305 , 891
    test_T010305 = test_T010305.drop([col], axis=1)

print('All NaN Column Drop.')

All NaN Check.
All NaN Column Drop.


In [46]:
# 각자 제일 많이 겹치는 LINE 2개씩 묶는데 안겹치는 컬럼 찾고 
list_train_T05 = list(set(train_T050304).difference(train_T050307))
list_train_T10 = list(set(train_T100304).difference(train_T100306))
list_train_T01 = list(set(train_T010306).difference(train_T010305))

In [47]:
# 그리고 일단 DROP!!!!! 이부분은 다시 확인해보자 유의한 컬럼 떨궜을수도 있음 ,,
for col in list_train_T05:
    try:
        train_T050304 = train_T050304.drop([col], axis=1)
        test_T050304 = test_T050304.drop([col], axis=1)
    except:
        pass
    try:
        train_T050307 = train_T050307.drop([col], axis=1)
        test_T050307 = test_T050307.drop([col], axis=1)
    except:
        pass

for col in list_train_T10:
    try:
        train_T100304 = train_T100304.drop([col], axis=1)
        test_T100304 = test_T100304.drop([col], axis=1)
    except:
        pass
    try:
        train_T100306 = train_T100306.drop([col], axis=1)
        test_T100306 = test_T100306.drop([col], axis=1)
    except:
        pass

for col in list_train_T01:
    try:
        train_T010306 = train_T010306.drop([col], axis=1)
        test_T010306 = test_T010306.drop([col], axis=1)
    except:
        pass
    try:
        train_T010305 = train_T010305.drop([col], axis=1)
        test_T010305 = test_T010305.drop([col], axis=1)
    except:
        pass

In [48]:
# 맞춘 친구들끼리 묶고
train_T05 = pd.concat([train_T050304, train_T050307])
train_T10 = pd.concat([train_T100304, train_T100306])
train_T01 = pd.concat([train_T010306, train_T010305])

test_T05 = pd.concat([test_T050304, test_T050307])
test_T10 = pd.concat([test_T100304, test_T100306])
test_T01 = pd.concat([test_T010306, test_T010305])

In [49]:
# 이제 분리했으니 사이좋게 레이블도 때고
train_y1 = train_T05['Y_Class']
train_y2 = train_T10['Y_Class']
train_y3 = train_T01['Y_Class']

train_T05 = train_T05.drop(['Y_Class'], axis=1)
train_T10 = train_T10.drop(['Y_Class'], axis=1)
train_T01 = train_T01.drop(['Y_Class'], axis=1)

In [50]:
## 레이블인코더로 인코딩
le1 = LabelEncoder()
for x in [train_T01, train_T05, train_T10]:
    le1 = le1.fit(x['PRODUCT_CODE'])
    x['PRODUCT_CODE'] = le1.transform(x['PRODUCT_CODE'])
        
for t in [test_T01, test_T05, test_T10]:
    for label in np.unique(t['PRODUCT_CODE']): 
        if label not in le1.classes_: 
            le1.classes_ = np.append(le1.classes_, label)
    t['PRODUCT_CODE'] = le1.transform(t['PRODUCT_CODE'])
print('CODE encode Done.')

le2 = LabelEncoder()
for x in [train_T01, train_T05, train_T10]:
    le2 = le2.fit(x['LINE'])
    x['LINE'] = le2.transform(x['LINE'])
        
for t in [test_T01, test_T05, test_T10]:
    for label in np.unique(t['LINE']): 
        if label not in le2.classes_: 
            le2.classes_ = np.append(le2.classes_, label)
    t['LINE'] = le2.transform(t['LINE'])
print('LINE encod Done.')

CODE encode Done.
LINE encod Done.


In [51]:
## 프로덕트 각 key인 이름 제거
train_T05 = train_T05.drop(['PRODUCT_ID'], axis=1)
train_T10 = train_T10.drop(['PRODUCT_ID'], axis=1)
train_T01 = train_T01.drop(['PRODUCT_ID'], axis=1)
print('Drop PRODUCT_ID')

Drop PRODUCT_ID


In [52]:
# train_T10 : 결측 대치 하고 , 그래도 남은 애들 세보기
coldict_T10 = {}
nalist= []

for col in train_T10.columns:
    if pd.isna(train_T10[col]).sum() <= len(train_T10) * 0.1:
        train_T10[col] = train_T10[col].fillna(np.nanmedian(train_T10[col])) # row*0.1 개 이하면 중앙값으로 결측 대치
        try:
            test_T10[col] = test_T10[col].fillna(np.nanmedian(train_T10[col]))
        except:
            pass
print('Done Fillna. (T10)')

for col in train_T10.columns:
    if pd.isna(train_T10[col]).sum() > 0:
        nalist.append(col)
print('Check NaN list. (T10)')

Done Fillna. (T10)
Check NaN list. (T10)


In [53]:
# train_T01 : 결측 대치 하고 , 그래도 남은 애들 세보기
coldict_T01 = {}
nalist2 = []

for col in train_T01.columns:
    if pd.isna(train_T01[col]).sum() <= len(train_T01)*0.1:
        train_T01[col] = train_T01[col].fillna(np.nanmedian(train_T01[col])) # row*0.1 개 이하면 중앙값으로 결측 대치
        try:
            test_T01[col] = test_T01[col].fillna(np.nanmedian(train_T01[col]))
        except:
            pass
print('Done Fillna. (T01)')

for col in train_T01.columns:
    if pd.isna(train_T01[col]).sum() > 0:
        nalist2.append(col)
print('Check NaN list. (T01)')

Done Fillna. (T01)
Check NaN list. (T01)


In [54]:
# train_T05 : 결측 대치 하고 , 그래도 남은 애들 세보기
coldict_T05 = {}
nalist3 = []

for col in train_T05.columns:
    if pd.isna(train_T05[col]).sum() <= len(train_T05)*0.1:
        train_T05[col] = train_T05[col].fillna(np.nanmedian(train_T05[col])) # row*0.1 개 이하면 중앙값으로 결측 대치
        try:
            test_T05[col] = test_T05[col].fillna(np.nanmedian(train_T05[col]))
        except:
            pass
print('Done fillna. (T05)')

for col in train_T05.columns:
    if pd.isna(train_T05[col]).sum() > 0:
        nalist3.append(col)
print('Check NaN list. (T05)')

Done fillna. (T05)
Check NaN list. (T05)


In [55]:
## 결측값 만들려고 있는 애들끼리만 임시로 dataframe 만들기
tmp_T05 = pd.concat([train_T05[nalist3]], axis=1)
tmp_T01 = pd.concat([train_T01[nalist2]], axis=1)
tmp_T10 = pd.concat([train_T10[nalist]], axis=1)

tmp_test_T05 = pd.concat([test_T05[nalist3]], axis=1)
tmp_test_T01 = pd.concat([test_T01[nalist2]], axis=1)
tmp_test_T10 = pd.concat([test_T10[nalist]], axis=1)

In [56]:
# 이걸로 결측대치
imputer1 = IterativeImputer(random_state=37)
imputer2 = IterativeImputer(random_state=37)
imputer3 = IterativeImputer(random_state=37, max_iter=5)

trans_x1 = imputer1.fit_transform(tmp_T05)
trans_x2 = imputer2.fit_transform(tmp_T01)
trans_x3 = imputer3.fit_transform(tmp_T10)

test_trans_x1 = imputer1.transform(tmp_test_T05)
print('Imputer Done.')
test_trans_x2 = imputer2.transform(tmp_test_T01)
print('Imputer Done.')
test_trans_x3 = imputer3.transform(tmp_test_T10)
print('Imputer Done.')

Imputer Done.
Imputer Done.
Imputer Done.


In [57]:
## 결측 대치한 후에 바까주기

train_T05 = train_T05.drop(nalist3, axis=1)
train_T01 = train_T01.drop(nalist2, axis=1)
train_T10 = train_T10.drop(nalist, axis=1)

test_T05 = test_T05.drop(nalist3, axis=1)
test_T01 = test_T01.drop(nalist2, axis=1)
test_T10 = test_T10.drop(nalist, axis=1)

fill_T05 = pd.DataFrame(trans_x1, columns=tmp_T05.columns)
fill_T01 = pd.DataFrame(trans_x2, columns=tmp_T01.columns)
fill_T10 = pd.DataFrame(trans_x3, columns=tmp_T10.columns)

test_fill_T05 = pd.DataFrame(test_trans_x1, columns=tmp_T05.columns)
test_fill_T01 = pd.DataFrame(test_trans_x2, columns=tmp_T01.columns)
test_fill_T10 = pd.DataFrame(test_trans_x3, columns=tmp_T10.columns)

In [58]:
train_T05 = pd.concat([train_T05], ignore_index=True)
train_T01 = pd.concat([train_T01], ignore_index=True)
train_T10 = pd.concat([train_T10], ignore_index=True)

train_T05 = pd.concat([train_T05, fill_T05], axis=1)
train_T01 = pd.concat([train_T01, fill_T01], axis=1)
train_T10 = pd.concat([train_T10, fill_T10], axis=1)

In [59]:
test_T05 = pd.concat([test_T05], ignore_index=True)
test_T01 = pd.concat([test_T01], ignore_index=True)
test_T10 = pd.concat([test_T10], ignore_index=True)

test_T05 = pd.concat([test_T05, test_fill_T05], axis=1)
test_T01 = pd.concat([test_T01, test_fill_T01], axis=1)
test_T10 = pd.concat([test_T10, test_fill_T10], axis=1)

In [60]:
## 한가지 값만 있는 데이터 떼기
one_data_col_train_T05 = []
one_data_col_train_T10 = []
one_data_col_train_T01 = []

for col in train_T05.filter(regex='X').columns:
    if len(train_T05[col].unique().tolist()) ==1:
        one_data_col_train_T05.append(col)

for col in train_T10.filter(regex='X').columns:
    if len(train_T10[col].unique().tolist()) ==1:
        one_data_col_train_T10.append(col)

for col in train_T01.filter(regex='X').columns:
    if len(train_T01[col].unique().tolist()) ==1:
        one_data_col_train_T01.append(col)

In [61]:
## 합쳐보기
train_tmp1 = pd.DataFrame()
test_tmp1 = pd.DataFrame()

train_tmp2 = pd.DataFrame()
test_tmp2 = pd.DataFrame()

train_tmp3 = pd.DataFrame()
test_tmp3 = pd.DataFrame()

for col in one_data_col_train_T05:
    train_tmp1 = pd.concat([train_tmp1, train_T05[col]], axis=1)
    test_tmp1 = pd.concat([test_tmp1, test_T05[col]], axis=1)
    train_T05 = train_T05.drop([col], axis=1)
    test_T05 = test_T05.drop([col], axis=1)

for col in one_data_col_train_T10:
    train_tmp2 = pd.concat([train_tmp2, train_T10[col]], axis=1)
    test_tmp2 = pd.concat([test_tmp2, test_T10[col]], axis=1)
    train_T10 = train_T10.drop([col], axis=1)
    test_T10 = test_T10.drop([col], axis=1)

for col in one_data_col_train_T01:
    train_tmp3 = pd.concat([train_tmp3, train_T01[col]], axis=1)
    test_tmp3 = pd.concat([test_tmp3, test_T01[col]], axis=1)
    train_T01 = train_T01.drop([col], axis=1)
    test_T01 = test_T01.drop([col], axis=1)

In [62]:
## 합친 column 지우는 columns들과 치환
train_tmp_ = pd.concat([train_tmp1[train_tmp1.columns[0]]], axis=1)
test_tmp_ = pd.concat([test_tmp1[test_tmp1.columns[0]]], axis=1)

for col in train_tmp1.columns[1:]:
    train_tmp_[train_tmp_.columns[0]] = train_tmp_[train_tmp_.columns[0]] + train_tmp1[col]
for col in test_tmp1.columns[1:]:
    test_tmp_[test_tmp_.columns[0]] = test_tmp_[test_tmp_.columns[0]] + test_tmp1[col]
    
train_T05 = pd.concat([train_T05, train_tmp_], axis=1)
test_T05 = pd.concat([test_T05, test_tmp_], axis=1)

In [63]:
train_tmp__ = pd.concat([train_tmp2[train_tmp2.columns[0]]], axis=1)
test_tmp__ = pd.concat([test_tmp2[test_tmp2.columns[0]]], axis=1)

for col in train_tmp2.columns[1:]:
    train_tmp__[train_tmp__.columns[0]] = train_tmp__[train_tmp__.columns[0]] + train_tmp2[col]
for col in test_tmp2.columns[1:]:
    test_tmp__[test_tmp__.columns[0]] = test_tmp__[test_tmp__.columns[0]] + test_tmp2[col]

train_T10 = pd.concat([train_T10, train_tmp__], axis=1)
test_T10 = pd.concat([test_T10, test_tmp__], axis=1)

In [64]:
train_tmp___ = pd.concat([train_tmp3[train_tmp3.columns[0]]], axis=1)
test_tmp___ = pd.concat([test_tmp3[test_tmp3.columns[0]]], axis=1)

for col in train_tmp3.columns[1:]:
    train_tmp___[train_tmp___.columns[0]] = train_tmp___[train_tmp___.columns[0]] + train_tmp3[col]
for col in test_tmp3.columns[1:]:
    test_tmp___[test_tmp___.columns[0]] = test_tmp___[test_tmp___.columns[0]] + test_tmp3[col]

train_T01 = pd.concat([train_T01, train_tmp___], axis=1)
test_T01 = pd.concat([test_T01, test_tmp___], axis=1)

In [65]:
## Y_Quality 제거, PRODUCT_ID 제거
train_quailty1 = train_T05['Y_Quality']
train_quailty2 = train_T10['Y_Quality']
train_quailty3 = train_T01['Y_Quality']

test_name1 = test_T05['PRODUCT_ID']
test_name2 = test_T10['PRODUCT_ID']
test_name3 = test_T01['PRODUCT_ID']

test_T05 = test_T05.drop(['PRODUCT_ID'], axis=1)
test_T10 = test_T10.drop(['PRODUCT_ID'], axis=1)
test_T01 = test_T01.drop(['PRODUCT_ID'], axis=1)

In [66]:
a = pd.DataFrame(train_y1)
a = pd.concat([a], ignore_index=True)
int_T05 = pd.concat([train_T05, a], axis=1)

a = pd.DataFrame(train_y2)
a = pd.concat([a], ignore_index=True)
int_T10 = pd.concat([train_T10, a], axis=1)

a = pd.DataFrame(train_y3)
a = pd.concat([a], ignore_index=True)
int_T01 = pd.concat([train_T01, a], axis=1)

In [67]:
# val셋 제작
val_T05_0 = int_T05[int_T05['Y_Class']==0][:8] # 41
val_T05_1 = int_T05[int_T05['Y_Class']==1][:9] # 46
val_T05_2 = int_T05[int_T05['Y_Class']==2][:6] # 33
val_T05 = pd.concat([val_T05_0, val_T05_1, val_T05_2]) # 120

val_T10_0 = int_T10[int_T10['Y_Class']==0][:5] # 28
val_T10_1 = int_T10[int_T10['Y_Class']==1][:57] # 289
val_T10_2 = int_T10[int_T10['Y_Class']==2][:6] # 32
val_T10 = pd.concat([val_T10_0, val_T10_1, val_T10_2]) # 349

val_T01_0 = int_T01[int_T01['Y_Class']==0][:4] # 19
val_T01_1 = int_T01[int_T01['Y_Class']==1][:14] # 72
val_T01_2 = int_T01[int_T01['Y_Class']==2][:7] # 38
val_T01 = pd.concat([val_T01_0, val_T01_1, val_T01_2]) # 129

# 다시 고치기
train_T05_ = pd.concat([int_T05[int_T05['Y_Class']==0][8:], int_T05[int_T05['Y_Class']==1][9:], int_T05[int_T05['Y_Class']==2][6:]], ignore_index=True)
train_T10_ = pd.concat([int_T10[int_T10['Y_Class']==0][5:], int_T10[int_T10['Y_Class']==1][57:], int_T10[int_T10['Y_Class']==2][6:]], ignore_index=True)
train_T01_ = pd.concat([int_T01[int_T01['Y_Class']==0][4:], int_T01[int_T01['Y_Class']==1][14:], int_T01[int_T01['Y_Class']==2][7:]], ignore_index=True)

In [68]:
# Y 떼기
train_y1_ = train_T05_['Y_Class']
train_T05_ = train_T05_.drop(['Y_Class'], axis=1)
train_y2_ = train_T10_['Y_Class']
train_T10_ = train_T10_.drop(['Y_Class'], axis=1)
train_y3_ = train_T01_['Y_Class']
train_T01_ = train_T01_.drop(['Y_Class'], axis=1)

val_y1 = val_T05['Y_Class']
val_T05 = val_T05.drop(['Y_Class'], axis=1)
val_y2 = val_T10['Y_Class']
val_T10 = val_T10.drop(['Y_Class'], axis=1)
val_y3 = val_T01['Y_Class']
val_T01 = val_T01.drop(['Y_Class'], axis=1)

train_quailty1_ = train_T05_['Y_Quality']
train_quailty2_ = train_T10_['Y_Quality']
train_quailty3_ = train_T01_['Y_Quality']

train_T05_ = train_T05_.drop(['Y_Quality'], axis=1)
train_T10_ = train_T10_.drop(['Y_Quality'], axis=1)
train_T01_ = train_T01_.drop(['Y_Quality'], axis=1)

val_quality1 = val_T05['Y_Quality']
val_quality2 = val_T10['Y_Quality']
val_quality3 = val_T01['Y_Quality']

val_T05 = val_T05.drop(['Y_Quality'], axis=1)
val_T10 = val_T10.drop(['Y_Quality'], axis=1)
val_T01 = val_T01.drop(['Y_Quality'], axis=1)

train_T05 = train_T05.drop(['Y_Quality'], axis=1)
train_T10 = train_T10.drop(['Y_Quality'], axis=1)
train_T01 = train_T01.drop(['Y_Quality'], axis=1)

In [69]:
# val set 평가
RF1 = RandomForestRegressor(n_estimators=300, random_state=cfg.seed).fit(train_T05_, train_quailty1_)
RF2 = RandomForestRegressor(n_estimators=100, random_state=cfg.seed).fit(train_T10_, train_quailty2_)
RF3 = RandomForestRegressor(n_estimators=150, random_state=cfg.seed).fit(train_T01_, train_quailty3_)

val_score1 = RF1.score(val_T05, val_quality1)
val_score2 = RF2.score(val_T10, val_quality2)
val_score3 = RF3.score(val_T01, val_quality3)

print(f'{RF1.score(train_T05_, train_quailty1_)}/{RF2.score(train_T10_, train_quailty2_)}/{RF3.score(train_T01_, train_quailty3_)}')
print(f'{val_score1}/{val_score2}/{val_score3}')

print('Val Regressed.')

0.9575475923191602/0.8984355274504304/0.9595075502518129
0.24136268025594054/-0.40425113941786983/0.23916134768960617
Val Regressed.


In [70]:
# 전체 train set 학습
RF1 = RandomForestRegressor(n_estimators=100, random_state=cfg.seed).fit(train_T05, train_quailty1)
RF2 = RandomForestRegressor(n_estimators=100, random_state=cfg.seed).fit(train_T10, train_quailty2)
RF3 = RandomForestRegressor(n_estimators=100, random_state=cfg.seed).fit(train_T01, train_quailty3)
print('Regressed.')

Regressed.


In [71]:
# test 평가
pred_quality1 = RF1.predict(test_T05)
pred_quality2 = RF2.predict(test_T10)
pred_quality3 = RF3.predict(test_T01)

for item in [pred_quality3, pred_quality2, pred_quality1]:
    item = item.reshape(-1,1)
    item2 = []
    for i in range(len(item)):
        item2.append(item[i][0])
    item = item2

train_quailty1_ = pd.concat([train_quailty1_], ignore_index=True)
train_quailty2_ = pd.concat([train_quailty2_], ignore_index=True)
train_quailty3_ = pd.concat([train_quailty3_], ignore_index=True)

train_T05['Y_Quality'] = train_quailty1
train_T10['Y_Quality'] = train_quailty2
train_T01['Y_Quality'] = train_quailty3

test_T05['Y_Quality'] = pred_quality1
test_T10['Y_Quality'] = pred_quality2
test_T01['Y_Quality'] = pred_quality3

print('Done!')

Done!


In [72]:
## Y_Quality 추가
train_T05_['Y_Quality'] = train_quailty1_
train_T10_['Y_Quality'] = train_quailty2_
train_T01_['Y_Quality'] = train_quailty3_

val_T05['Y_Quality'] = val_quality1
val_T10['Y_Quality'] = val_quality2
val_T01['Y_Quality'] = val_quality3

In [73]:
# train_y1.value_counts() # 41/46/33 {0:1.0, 1:1.0, 2:1.0}
# train_y2.value_counts() # 28/289/32 {0:1.5, 1:1.0, 2:1.5}
# train_y3.value_counts() # 19/72/38 {0:1.2, 1:1.0, 2:1.2}

# classification val set 평가
RFC1 = RandomForestClassifier(n_estimators=300, max_depth=5, random_state=cfg.seed, class_weight={0:1.2, 1:1.0, 2:1.2}).fit(train_T05_, train_y1_)
RFC2 = RandomForestClassifier(n_estimators=100, random_state=cfg.seed, class_weight={0:1.2, 1:1.0, 2:1.2}).fit(train_T10_, train_y2_)
RFC3 = RandomForestClassifier(n_estimators=150,  random_state=cfg.seed, class_weight={0:1.2, 1:1.0, 2:1.2}).fit(train_T01_, train_y3_)

val_score1 = RFC1.score(val_T05, val_y1)
val_score2 = RFC2.score(val_T10, val_y2)
val_score3 = RFC3.score(val_T01, val_y3)

print(f'{RFC1.score(train_T05_, train_y1_)}/{RFC2.score(train_T10_, train_y2_)}/{RFC3.score(train_T01_, train_y3_)}')
print(f'{val_score1}/{val_score2}/{val_score3}')

print('Val Classified.')

1.0/1.0/1.0
0.391304347826087/0.8382352941176471/0.68
Val Classified.


In [74]:
# weights = {0:1.2, 1:1.0, 2:1.2}
RFC1 = RandomForestClassifier(n_estimators=300, max_depth=5, random_state=cfg.seed, class_weight={0:1.2, 1:1.0, 2:1.2}).fit(train_T05, train_y1)
RFC2 = RandomForestClassifier(n_estimators=100, random_state=cfg.seed, class_weight={0:1.2, 1:1.0, 2:1.2}).fit(train_T10, train_y2)
RFC3 = RandomForestClassifier(n_estimators=150,  random_state=cfg.seed, class_weight={0:1.2, 1:1.0, 2:1.2}).fit(train_T01, train_y3)

print('Classified.')

Classified.


In [75]:
preds1 = RFC1.predict(test_T05)
preds2 = RFC2.predict(test_T10)
preds3 = RFC3.predict(test_T01)

preds1 = pd.DataFrame(preds1, columns=['Y_Class'])
preds2 = pd.DataFrame(preds2, columns=['Y_Class'])
preds3 = pd.DataFrame(preds3, columns=['Y_Class'])

In [76]:
a = pd.concat([test_name1], ignore_index=True)
b = pd.concat([test_name2], ignore_index=True)
c = pd.concat([test_name3], ignore_index=True)

preds1 = pd.concat([a, preds1], axis=1)
preds2 = pd.concat([b, preds2], axis=1)
preds3 = pd.concat([c, preds3], axis=1)

In [77]:
preds = pd.concat([preds1, preds2, preds3])
preds = pd.concat([preds], ignore_index=True).sort_values('PRODUCT_ID', ignore_index=True).drop(['PRODUCT_ID'], axis=1)

In [78]:
submit = pd.read_csv('./data/sample_submission.csv')
submit['Y_Class'] = preds
print(f'{len(submit[submit["Y_Class"]==0])}/{len(submit[submit["Y_Class"]==1])}/{len(submit[submit["Y_Class"]==2])}')
submit.to_csv('./baseline_submission_split_최종.csv', index=False)

39/264/7
